In [1]:
DF_PATH = "C:/Users/M/Documents/data_science_project/data/processed/2_cleaned_data.pkl"

ROLE_COLS = ['DevType']
TECH_COLS = ['LanguageWorkedWith','DatabaseWorkedWith','WebframeWorkedWith','MiscTechWorkedWith']

EXPERIMENT_NAME = "stackoverflow_single_model"
LOG_PATH = "C:/Users/M/Documents/data_science_project/models/temp/random_forest/"
LOG_DATA_PKL =  "data.pkl"
LOG_MODEL_PKL =  "model.pkl"
LOG_METRICS_PKL =  "metrics.pkl"

In [2]:
# Load packages
import pandas as pd 
import numpy as np
import logging
import pickle
import random
import plotly 
import os

import mlflow
from mlflow.tracking import MlflowClient

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.feature_selection import VarianceThreshold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import auc, accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

from sklearn.decomposition import PCA, KernelPCA

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from matplotlib import pyplot as plt

In [3]:
df = pd.read_pickle(DF_PATH)
df

LanguageWorkedWith                                                   \
                Assembly Bash/Shell/PowerShell  C C# C++ Dart Go HTML/CSS   
0                      0                     0  0  1   0    0  0        1   
1                      0                     0  0  0   0    0  0        0   
5                      0                     0  0  0   0    0  0        1   
8                      0                     0  0  0   0    0  0        1   
11                     0                     0  0  0   0    0  0        1   
...                  ...                   ... .. ..  ..  ... ..      ...   
64405                  0                     0  0  0   0    0  0        0   
64416                  0                     0  0  0   0    0  0        0   
64422                  0                     0  0  0   0    0  0        1   
64428                  0                     1  0  0   0    0  0        1   
64447                  0                     0  0  1   0    0  0        1   

                    ... skills_clusters                                 \
      Haskell Java  ... skills_group_15 skills_group_16 skills_group_2   
0           0    0  ...               0               0              0   
1           0    0  ...               0               1              0   
5           0    1  ...               0               0              0   
8           0    0  ...               0               0              1   
11          0    0  ...               0               0              0   
...       ...  ...  ...             ...             ...            ...   
64405       0    1  ...               0               0              1   
64416       0    1  ...               0               0              0   
64422       0    0  ...               0               0              2   
64428       0    0  ...               0               0              4   
64447       0    1  ...               0               1              2   

                                                                   \
      skills_group_3 skills_group_4 skills_group_5 skills_group_6   
0                  0              0              0              0   
1                  0              1              0              0   
5                  0              0              0              0   
8                  0              0              0              0   
11                 0              0              0              0   
...              ...            ...            ...            ...   
64405              0              0              1              0   
64416              0              0              0              0   
64422              0              0              3              0   
64428              0              0              1              0   
64447              0              0              0              0   

                                                    
      skills_group_7 skills_group_8 skills_group_9  
0                  0              0              1  
1                  0              0              0  
5                  0              0              1  
8                  0              0              0  
11                 0              0              0  
...              ...            ...            ...  
64405              0              0              1  
64416              0              0              2  
64422              0              0              0  
64428              0              0              0  
64447              0              0              2  

[16665 rows x 107 columns]

In [4]:
def calculate_quality(ground_truth ,prediction , metric ,sort_values = False) :
    quality_scores = {}
    for col in predictions.columns :
        role_pred = predictions[col].copy()
        role_truth = ground_truth[col].copy()
        quality_scores[col] = round(metric(role_truth ,role_pred)*100 ,2)
    quality_scores = pd.Series(quality_scores.values() ,index = quality_scores.keys())
    if sort_values :
        quality_scores = quality_scores.sort_values()
    return quality_scores

In [5]:
role_df = df['DevType'].copy()
role_sum = role_df.sum(axis = 0)
role_sum

Academic researcher                               581
Data or business analyst                          669
Data scientist or machine learning specialist     799
Database administrator                            296
DevOps specialist                                 677
Developer, QA or test                             493
Developer, back-end                              5503
Developer, desktop or enterprise applications    1671
Developer, embedded applications or devices       795
Developer, front-end                             2890
Developer, full-stack                            5578
Developer, game or graphics                       342
Developer, mobile                                1859
Engineer, data                                    483
Scientist                                         292
System administrator                              440
dtype: int64

In [6]:
samples_per_class = 500
resampled_roles = []
for role_col in role_df.columns :
    sub_df = role_df.loc[role_df[role_col] == 1].copy()
    if len(sub_df) < samples_per_class :
        # upsample
        sub_df = sub_df.sample(samples_per_class ,replace = True ,random_state = 0)
    else : 
        # downsample
        sub_df = sub_df.sample(samples_per_class ,random_state = 0)
        
    resampled_roles.append(sub_df)

In [7]:
roles_df = pd.concat(resampled_roles)
roles_df.sum(0)

Academic researcher                               797
Data or business analyst                          767
Data scientist or machine learning specialist     871
Database administrator                            588
DevOps specialist                                 705
Developer, QA or test                             590
Developer, back-end                              1770
Developer, desktop or enterprise applications     892
Developer, embedded applications or devices       666
Developer, front-end                              893
Developer, full-stack                            1431
Developer, game or graphics                       560
Developer, mobile                                 800
Engineer, data                                    631
Scientist                                         642
System administrator                              647
dtype: int64

In [8]:
df = df.loc[roles_df.index]
df

LanguageWorkedWith                                                   \
                Assembly Bash/Shell/PowerShell  C C# C++ Dart Go HTML/CSS   
54451                  0                     0  0  0   0    0  0        1   
37550                  0                     0  0  1   1    0  0        0   
49646                  0                     1  1  0   1    0  0        0   
47422                  0                     0  0  0   0    0  0        0   
5139                   0                     0  1  0   1    0  0        1   
...                  ...                   ... .. ..  ..  ... ..      ...   
53453                  0                     1  0  0   0    0  0        0   
44375                  0                     1  1  0   0    0  0        0   
15982                  0                     1  0  0   0    0  0        1   
63800                  0                     0  0  0   0    0  0        1   
49466                  0                     0  0  0   0    0  0        1   

                    ... skills_clusters                                 \
      Haskell Java  ... skills_group_15 skills_group_16 skills_group_2   
54451       0    0  ...               0               0              0   
37550       0    0  ...               0               0              0   
49646       0    0  ...               0               0              2   
47422       0    0  ...               0               0              1   
5139        0    1  ...               0               0              2   
...       ...  ...  ...             ...             ...            ...   
53453       0    0  ...               0               0              2   
44375       0    1  ...               0               0              2   
15982       0    0  ...               0               0              0   
63800       0    1  ...               0               0              2   
49466       0    0  ...               0               0              3   

                                                                   \
      skills_group_3 skills_group_4 skills_group_5 skills_group_6   
54451              0              4              0              0   
37550              0              0              1              0   
49646              0              0              3              0   
47422              0              0              1              0   
5139               0              0              2              0   
...              ...            ...            ...            ...   
53453              0              0              1              0   
44375              0              0              2              0   
15982              0              0              1              0   
63800              0              0              2              0   
49466              0              0              1              0   

                                                    
      skills_group_7 skills_group_8 skills_group_9  
54451              0              2              0  
37550              0              0              0  
49646              1              0              1  
47422              0              0              0  
5139               0              0              1  
...              ...            ...            ...  
53453              0              0              0  
44375              0              0              1  
15982              0              0              0  
63800              0              0              1  
49466              0              0              0  

[8000 rows x 107 columns]

In [9]:
X_train ,X_test ,y_train ,y_test = train_test_split(df.drop('DevType' ,axis = 1) ,df['DevType'] ,random_state = 0)

C:\Users\M\anaconda3\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [10]:
#Initialze ml flow
client = MlflowClient()
mlflow.set_experiment(EXPERIMENT_NAME)
exp = client.get_experiment_by_name(EXPERIMENT_NAME)

## 1.Vanilla Forest

In [11]:
rf_clf = make_pipeline(StandardScaler() ,
                      RandomForestClassifier(random_state=0))

In [12]:
rf_clf.fit(X_train ,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=0))])

In [13]:
# Evaluate on train set
predictions = pd.DataFrame(rf_clf.predict(X_train) ,columns = y_train.columns)
train_scores = {score.__name__ : calculate_quality(y_train ,predictions ,score) 
               for score in [accuracy_score ,precision_score ,recall_score ,f1_score]}
train_scores = pd.concat(train_scores ,axis = 1)
train_scores

,accuracy_score,precision_score,recall_score,f1_score
Academic researcher,98.90,96.95,91.69,94.25
Data or business analyst,99.37,97.44,96.12,96.77
Data scientist or machine learning specialist,99.25,97.26,95.94,96.59
Database administrator,99.32,94.49,96.40,95.44
DevOps specialist,99.58,99.41,95.83,97.59
"Developer, QA or test",99.42,98.52,93.22,95.80
"Developer, back-end",98.68,98.82,95.17,96.96
"Developer, desktop or enterprise applications",98.72,97.61,90.80,94.08
"Developer, embedded applications or devices",98.82,91.84,94.22,93.02
"Developer, front-end",99.47,98.23,97.09,97.66


In [14]:
# Evaluate on test set
predictions = pd.DataFrame(rf_clf.predict(X_test) ,columns = y_test.columns)
test_scores = {score.__name__ : calculate_quality(y_test ,predictions ,score) 
              for score in [accuracy_score ,precision_score ,recall_score ,f1_score]}
test_scores = pd.concat(test_scores ,axis = 1)
test_scores

,accuracy_score,precision_score,recall_score,f1_score
Academic researcher,94.05,83.85,52.66,64.69
Data or business analyst,95.10,83.33,54.60,65.97
Data scientist or machine learning specialist,95.30,81.82,69.90,75.39
Database administrator,97.10,84.00,73.43,78.36
DevOps specialist,95.15,90.00,50.85,64.98
"Developer, QA or test",96.25,94.85,56.79,71.04
"Developer, back-end",85.20,79.07,45.74,57.95
"Developer, desktop or enterprise applications",90.95,70.79,28.90,41.04
"Developer, embedded applications or devices",94.30,73.15,48.17,58.09
"Developer, front-end",93.30,74.00,53.88,62.36


In [15]:
mean_test_scores = test_scores.mean()
mean_test_scores

accuracy_score     94.196875
precision_score    83.858750
recall_score       57.422500
f1_score           67.671250
dtype: float64

## Log

In [16]:
# Data details
data_details = {"data_path": DF_PATH,
                "training_indices": X_train.index.tolist(),
                "test_indices":     X_test.index.tolist(), 
                "features_names":   X_train.columns.droplevel(0).tolist(),
                "targets_names":    y_train.columns.tolist()}

with open(os.path.join(LOG_PATH, LOG_DATA_PKL), "wb") as output_file:
    pickle.dump(data_details, output_file)

In [17]:
# Model
model = {"model_description": "Random Forest: naive",
         "model_details": str(rf_clf),
         "model_object": rf_clf} 

with open(os.path.join(LOG_PATH, LOG_MODEL_PKL), "wb") as output_file:
    pickle.dump(model, output_file)

In [18]:
# Preformance details
classes_metrics = {"train_scores": train_scores, 
                   "test_scores":  test_scores}

with open(os.path.join(LOG_PATH, LOG_METRICS_PKL), "wb") as output_file:
    pickle.dump(classes_metrics, output_file)

In [19]:
# Start a new run and track 
with mlflow.start_run(experiment_id=exp.experiment_id, 
                      run_name=model["model_description"]):
    # Log pickles
    mlflow.log_artifacts(LOG_PATH)  
    
    # Track metrics 
    for metric, score in mean_test_scores.items():
        mlflow.log_metric(metric, score) 

## 2.Random Forest With Non-Linearity

In [20]:
rf_clf = make_pipeline(StandardScaler(), 
                       FeatureUnion([('linear_pca', PCA(n_components=40)), 
                                      ('kernel_pca', KernelPCA(n_components=40, 
                                                               kernel='rbf'))]),
                       RandomForestClassifier(random_state=0))

rf_clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('featureunion',
                 FeatureUnion(transformer_list=[('linear_pca',
                                                 PCA(n_components=40)),
                                                ('kernel_pca',
                                                 KernelPCA(kernel='rbf',
                                                           n_components=40))])),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=0))])

In [21]:
# Evaluate on train set
predictions = pd.DataFrame(rf_clf.predict(X_train) ,columns = y_train.columns)
train_scores = {score.__name__ : calculate_quality(y_train ,predictions ,score) 
               for score in [accuracy_score ,precision_score ,recall_score ,f1_score]}
train_scores = pd.concat(train_scores ,axis = 1)
train_scores

,accuracy_score,precision_score,recall_score,f1_score
Academic researcher,98.83,98.15,89.83,93.81
Data or business analyst,99.37,97.44,96.12,96.77
Data scientist or machine learning specialist,99.27,97.26,96.09,96.67
Database administrator,99.32,94.69,96.18,95.43
DevOps specialist,99.58,99.22,96.02,97.59
"Developer, QA or test",99.40,99.25,92.29,95.64
"Developer, back-end",98.65,99.13,94.71,96.87
"Developer, desktop or enterprise applications",98.68,98.69,89.47,93.85
"Developer, embedded applications or devices",98.80,92.49,93.23,92.86
"Developer, front-end",99.45,98.52,96.65,97.58


In [22]:
# Evaluate on test set
predictions = pd.DataFrame(rf_clf.predict(X_test) ,columns = y_test.columns)
test_scores = {score.__name__ : calculate_quality(y_test ,predictions ,score) 
              for score in [accuracy_score ,precision_score ,recall_score ,f1_score]}
test_scores = pd.concat(test_scores ,axis = 1)
test_scores

,accuracy_score,precision_score,recall_score,f1_score
Academic researcher,94.10,92.38,46.86,62.18
Data or business analyst,95.05,86.41,51.15,64.26
Data scientist or machine learning specialist,95.05,84.52,63.59,72.58
Database administrator,97.40,88.24,73.43,80.15
DevOps specialist,95.15,93.48,48.59,63.94
"Developer, QA or test",96.30,95.83,56.79,71.32
"Developer, back-end",84.10,74.06,44.17,55.34
"Developer, desktop or enterprise applications",91.70,90.62,26.61,41.13
"Developer, embedded applications or devices",94.10,75.56,41.46,53.54
"Developer, front-end",93.35,80.17,47.09,59.33


In [23]:
mean_test_scores = test_scores.mean()
mean_test_scores

accuracy_score     94.190625
precision_score    87.516875
recall_score       54.773750
f1_score           66.562500
dtype: float64

## Log

In [24]:
# Data details
data_details = {"data_path": DF_PATH,
                "training_indices": X_train.index.tolist(),
                "test_indices":     X_test.index.tolist(), 
                "features_names":   X_train.columns.droplevel(0).tolist(),
                "targets_names":    y_train.columns.tolist()}

with open(os.path.join(LOG_PATH, LOG_DATA_PKL), "wb") as output_file:
    pickle.dump(data_details, output_file)

In [25]:
# Model
model = {"model_description": "Random Forest: with non-linearity",
         "model_details": str(rf_clf),
         "model_object": rf_clf} 

with open(os.path.join(LOG_PATH, LOG_MODEL_PKL), "wb") as output_file:
    pickle.dump(model, output_file)

In [26]:
# Preformance details
classes_metrics = {"train_scores": train_scores, 
                   "test_scores":  test_scores}

with open(os.path.join(LOG_PATH, LOG_METRICS_PKL), "wb") as output_file:
    pickle.dump(classes_metrics, output_file)

In [27]:
# Start a new run and track 
with mlflow.start_run(experiment_id=exp.experiment_id, 
                      run_name=model["model_description"]):
    # Log pickles
    mlflow.log_artifacts(LOG_PATH)  
    
    # Track metrics 
    for metric, score in mean_test_scores.items():
        mlflow.log_metric(metric, score) 

## 3.Random Forest With PCAs And Hyper parameter Tuning

In [28]:
hpt_rf_clf = make_pipeline(StandardScaler(), 
                           FeatureUnion([('linear_pca', PCA()), 
                                         ('kernel_pca', KernelPCA(kernel='rbf'))]),
                           RandomForestClassifier(random_state=0, ))

In [29]:
# list(hpt_rf_clf.get_params().keys())
tuned_parameters = [{'featureunion__linear_pca__n_components': [5, 10, 20, 40, 60], 
                     'featureunion__kernel_pca__n_components': [5, 10, 20, 40, 60],
                     'randomforestclassifier__n_estimators':   [100, 500, 1000]}]

In [ ]:
hpt_rf_clf = GridSearchCV(hpt_rf_clf, 
                          tuned_parameters, 
                          n_jobs=-1)

hpt_rf_clf.fit(X_train, y_train)

In [ ]:
hpt_rf_clf.best_params_

In [ ]:
# Evaluate on train set
predictions = pd.DataFrame(hpt_rf_clf.predict(X_train) ,columns = y_train.columns)
train_scores = {score.__name__ : calculate_quality(y_train ,predictions ,score) 
               for score in [accuracy_score ,precision_score ,recall_score ,f1_score]}
train_scores = pd.concat(train_scores ,axis = 1)
train_scores

In [ ]:
# Evaluate on test set
predictions = pd.DataFrame(hpt_rf_clf.predict(X_test) ,columns = y_test.columns)
test_scores = {score.__name__ : calculate_quality(y_test ,predictions ,score) 
              for score in [accuracy_score ,precision_score ,recall_score ,f1_score]}
test_scores = pd.concat(test_scores ,axis = 1)
test_scores

In [88]:
mean_test_scores = test_scores.mean()
mean_test_scores

accuracy_score     94.212500
precision_score    87.551875
recall_score       54.653125
f1_score           66.531875
dtype: float64

In [ ]:
# Data details
data_details = {"data_path": DF_PATH,
                "training_indices": X_train.index.tolist(),
                "test_indices":     X_test.index.tolist(), 
                "features_names":   X_train.columns.droplevel(0).tolist(),
                "targets_names":    y_train.columns.tolist()}

with open(os.path.join(LOG_PATH, LOG_DATA_PKL), "wb") as output_file:
    pickle.dump(data_details, output_file)

In [ ]:
# Model
model = {"model_description": "Random Forest: with non-linearity",
         "model_details": str(rf_clf),
         "model_object": rf_clf} 

with open(os.path.join(LOG_PATH, LOG_MODEL_PKL), "wb") as output_file:
    pickle.dump(model, output_file)

In [ ]:
# Preformance details
classes_metrics = {"train_scores": train_scores, 
                   "test_scores":  test_scores}

with open(os.path.join(LOG_PATH, LOG_METRICS_PKL), "wb") as output_file:
    pickle.dump(classes_metrics, output_file)

In [ ]:
# Start a new run and track 
with mlflow.start_run(experiment_id=exp.experiment_id, 
                      run_name=model["model_description"]):
    # Log pickles
    mlflow.log_artifacts(LOG_PATH)  
    
    # Track metrics 
    for metric, score in mean_test_scores.items():
        mlflow.log_metric(metric, score) 

In [38]:
runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.accuracy_score,metrics.precision_score,metrics.recall_score,metrics.f1_score,tags.mlflow.user,tags.mlflow.source.name,tags.mlflow.runName,tags.mlflow.source.type
0,67403f4417c34d82b102947d9e1aaa9d,1,FINISHED,file:///C:/Users/M/Documents/data_science_proj...,2021-11-17 13:31:59.886000+00:00,2021-11-17 13:32:04.446000+00:00,94.190625,87.516875,54.773750,66.562500,M,C:\Users\M\anaconda3\lib\site-packages\ipykern...,Random Forest: with non-linearity,LOCAL
1,2e891e11a7614520bf10200131b6c185,1,FINISHED,file:///C:/Users/M/Documents/data_science_proj...,2021-11-17 13:30:34.257000+00:00,2021-11-17 13:30:37.581000+00:00,94.196875,83.858750,57.422500,67.671250,M,C:\Users\M\anaconda3\lib\site-packages\ipykern...,Random Forest: naive,LOCAL
2,2e83639e11174047842ef131ac7d695d,1,FINISHED,file:///C:/Users/M/Documents/data_science_proj...,2021-11-17 12:58:34.422000+00:00,2021-11-17 12:58:38.335000+00:00,94.212500,87.052500,54.775000,66.573125,M,C:\Users\M\anaconda3\lib\site-packages\ipykern...,Random Forest: with non-linearity,LOCAL
3,6ed653153b824154bde0bc3acb66766f,1,FINISHED,file:///C:/Users/M/Documents/data_science_proj...,2021-11-17 12:57:01.421000+00:00,2021-11-17 12:57:06.483000+00:00,94.196875,83.858750,57.422500,67.671250,M,C:\Users\M\anaconda3\lib\site-packages\ipykern...,Random Forest: naive,LOCAL
4,fa4ee3ab220a45ae9ff1f8cfb20a6889,1,FINISHED,file:///C:/Users/M/Documents/data_science_proj...,2021-11-17 00:25:19.392000+00:00,2021-11-17 00:25:22.464000+00:00,94.159375,86.764375,54.807500,66.520000,M,C:\Users\M\anaconda3\lib\site-packages\ipykern...,Random Forest: with non-linearity,LOCAL
5,3d4dd66ea106410a9168d512798dd8db,1,FINISHED,file:///C:/Users/M/Documents/data_science_proj...,2021-11-17 00:23:16.762000+00:00,2021-11-17 00:23:29.989000+00:00,94.196875,83.858750,57.422500,67.671250,M,C:\Users\M\anaconda3\lib\site-packages\ipykern...,Random Forest: naive,LOCAL
6,4b34551397524a408d92a983e5779fcd,1,FINISHED,file:///C:/Users/M/Documents/data_science_proj...,2021-11-16 23:18:58.048000+00:00,2021-11-16 23:19:00.445000+00:00,94.234375,87.313125,54.877500,66.680000,M,C:\Users\M\anaconda3\lib\site-packages\ipykern...,Random Forest: with non-linearity,LOCAL
7,83871869e617478193c02c2622f1542f,1,FINISHED,file:///C:/Users/M/Documents/data_science_proj...,2021-11-16 23:14:51.564000+00:00,2021-11-16 23:14:55.575000+00:00,94.196875,83.858750,57.422500,67.671250,M,C:\Users\M\anaconda3\lib\site-packages\ipykern...,Random Forest: naive,LOCAL
8,b37d1c480093457fb7c9a98ab3c70758,1,FINISHED,file:///C:/Users/M/Documents/data_science_proj...,2021-11-16 23:04:46.323000+00:00,2021-11-16 23:04:51.119000+00:00,94.228125,82.146250,59.588125,68.566250,M,C:\Users\M\anaconda3\lib\site-packages\ipykern...,Random Forest: naive,LOCAL
9,b8cc0c830c844e96ad9209073b90bec2,1,FINISHED,file:///C:/Users/M/Documents/data_science_proj...,2021-11-16 13:10:06.855000+00:00,2021-11-16 13:10:07.126000+00:00,91.190625,65.601250,28.439375,37.190625,M,C:\Users\M\anaconda3\lib\site-packages\ipykern...,Baseline Model : Logistic Regression,LOCAL
